In [1]:
import numpy as np
import os
import shutil

In [2]:
# rewrite_if_exist = True
rewrite_if_exist = False

# For case studies

In [16]:
destination_dir = "/projects/mcmm/results_final/"

In [3]:
# mcir may 16

In [4]:
mcir_dir = "/projects/mcmm/syn_lownodeexp/"

In [5]:
config_copied = []

for filename in os.listdir(mcir_dir):
    if not filename.endswith(".npy"):
        continue
    
    src = os.path.join(mcir_dir, filename)
    
    splitted_fn = filename.split("_")
    splitted_fn.insert(2, "mcir")
    splitted_fn[1] = splitted_fn[1][:-1]
    target_filename = "_".join(splitted_fn)
    dst = os.path.join(destination_dir, target_filename)
    
    config_copied.append((splitted_fn[0].lower(), int(splitted_fn[1])))
    
    if not (os.path.exists(dst) and not rewrite_if_exist):
        shutil.copy2(src, dst)

In [6]:
# mcir may 14

In [7]:
mcir_dirs = [
    "/projects/mcmm/benchmark_2023-05-13/Michalewicz_May12/",
    "/projects/mcmm/benchmark_2023-05-13/Levy_May12/",
    "/projects/mcmm/benchmark_2023-05-13/Ackley_May12/",
    "/projects/mcmm/benchmark_2023-05-13/nlp/",
]

In [8]:
for mcir_dir in mcir_dirs:
    print("Processing {}".format(mcir_dir))
    for filename in os.listdir(mcir_dir):
        if not filename.endswith(".npy"):
            continue
        
        filename_no_appendix = filename.strip(".npy")
        filename_splitted = filename_no_appendix.split("_")
        func = filename_splitted[0]
        dim = int(filename_splitted[1][:-1])
        config = (func.lower(), dim)
        if config in config_copied:
            continue
        
        target_filename_list = [func, str(dim), "mcir", *filename_splitted[2:], "Y"]
        target_filename = "_".join(target_filename_list) + ".npy"
        
        records_Y = np.load(os.path.join(mcir_dir, filename))[:, 0]
        
        dst = os.path.join(destination_dir, target_filename)
        if not (os.path.exists(dst) and not rewrite_if_exist):
            np.save(dst, records_Y)

Processing /projects/mcmm/benchmark_2023-05-13/Michalewicz_May12/
Processing /projects/mcmm/benchmark_2023-05-13/Levy_May12/
Processing /projects/mcmm/benchmark_2023-05-13/Ackley_May12/
Processing /projects/mcmm/benchmark_2023-05-13/nlp/


In [9]:
# baselines

In [10]:
baseline_dir_1 = "/projects/mcmm/running_zhizhen_exps/baseline_results_displacement/record"
baseline_dir_2 = "/projects/mcmm/running_zhizhen_exps/baseline_results_displacement_more_iterations/record"

In [11]:
result_files_1 = [fn for fn in os.listdir(baseline_dir_1) if fn.endswith("Y.npy")]
result_files_2 = [fn for fn in os.listdir(baseline_dir_2) if fn.endswith("Y.npy")]

In [12]:
files_in_both = []

for file_1 in result_files_1:
    if file_1 not in result_files_2:
        dst = os.path.join(destination_dir, file_1)
        if not (os.path.exists(dst) and not rewrite_if_exist):
            shutil.copy2(
                os.path.join(baseline_dir_1, file_1),
                dst
            )
    else:
        files_in_both.append(file_1)
    
for file_2 in result_files_2:
    if file_2 not in result_files_1:
        dst = os.path.join(destination_dir, file_2)
        if not (os.path.exists(dst) and not rewrite_if_exist):
            shutil.copy2(
                os.path.join(baseline_dir_2, file_2),
                dst,
            )

In [13]:
for file in files_in_both:
    path_1 = os.path.join(baseline_dir_1, file)
    path_2 = os.path.join(baseline_dir_2, file)
    
    array_1 = np.load(path_1)
    array_2 = np.load(path_2)
    
    assert len(array_1.shape) == 1 and len(array_2.shape) == 1
    
    if len(array_1) > len(array_2):
        src = path_1
    else:
        src = path_2
    
    dst = os.path.join(destination_dir, file)
    if not (os.path.exists(dst) and not rewrite_if_exist):
        shutil.copy2(src, dst)

In [14]:
# nlp baselines

In [15]:
baseline_dir_nlp = "/projects/mcmm/baselines_nlp/record/"

In [16]:
result_files = [fn for fn in os.listdir(baseline_dir_nlp) if fn.endswith("Y.npy")]

In [17]:
for file in result_files:
    dst = os.path.join(destination_dir, file)
    if not (os.path.exists(dst) and not rewrite_if_exist):
        shutil.copy2(
            os.path.join(baseline_dir_nlp, file),
            os.path.join(destination_dir, file)
        )

In [18]:
# MCIR Neural Network

In [19]:
mcir_dir_nn = "/projects/mcmm/benchmark_2023-05-13/nn_one_layer_May13"

In [20]:
def process_mcir_nn_filename(filename):
    # Example: nn_one_layer_michalewicz_100_16_100d_tfxjsg932mg9may2e8jrf0xhtxkt0ap6egfodzz1.npy
    filename_no_npy = filename.strip('.npy')
    splitted = filename_no_npy.split("_")
    simulated_func = splitted[3]
    input_dimension = int(splitted[4])
    hidden_dimension = int(splitted[5])
    identifier = splitted[-1]
    algorithm = 'mcir'
    
    new_file_name = '_'.join([
        f"nn-{simulated_func}",
        str(input_dimension),
        algorithm,
        identifier,
        "Y"
    ]) + ".npy"
    
    return new_file_name

In [21]:
process_mcir_nn_filename("nn_one_layer_michalewicz_100_16_100d_tfxjsg932mg9may2e8jrf0xhtxkt0ap6egfodzz1.npy")

'nn-michalewicz_100_mcir_tfxjsg932mg9may2e8jrf0xhtxkt0ap6egfodzz1_Y.npy'

In [22]:
for file in os.listdir(mcir_dir_nn):
    if not file.endswith(".npy"):
        continue
    
    src = os.path.join(mcir_dir_nn, file)
    new_filename = process_mcir_nn_filename(file)
    dst = os.path.join(destination_dir, new_filename)
    
    records_Y = np.load(src)[:, 0]
        
    if not (os.path.exists(dst) and not rewrite_if_exist):
        np.save(dst, records_Y)

In [23]:
# baseline Neural Network

In [24]:
baseline_dir_nn = "/projects/mcmm/running_zhizhen_exps/baseline_results_nn_2/record"

In [25]:
def process_baseline_nn_filename(filename):
    # Example: NeuralNetworkOneLayer_-1_scipy_dual_annealing_4-nn_one_layer_michalewicz_50_16_Y.npy
    filename_no_npy = filename.strip('.npy')
    
    nn_config = filename.rsplit('-', 1)[1]
    splitted_nn_config = nn_config.split('_')
    
    simulated_func = splitted_nn_config[3]
    input_dimension = int(splitted_nn_config[4])
    hidden_dimension = int(splitted_nn_config[5])
    
    run_info = filename.rsplit('-', 1)[0]
    splitted_run_info = run_info.split('_')
    
    seed = int(run_info.rsplit('_', 1)[1])
    identifier = str(seed)
    
    algorithm = run_info.rsplit('_', 1)[0].split('_', 3)[-1]
    
    new_file_name = '_'.join([
        f"nn-{simulated_func}",
        str(input_dimension),
        algorithm,
        identifier,
        "Y"
    ]) + ".npy"
    
    return new_file_name

In [26]:
process_baseline_nn_filename("NeuralNetworkOneLayer_-1_scipy_dual_annealing_4-nn_one_layer_michalewicz_50_16_Y.npy")

'nn-michalewicz_50_dual_annealing_4_Y.npy'

In [27]:
for file in os.listdir(baseline_dir_nn):
    if not file.endswith("_Y.npy"):
        continue
    
    src = os.path.join(baseline_dir_nn, file)
    new_filename = process_baseline_nn_filename(file)
    dst = os.path.join(destination_dir, new_filename)
    
    records_Y = np.load(src)
    
    if not (os.path.exists(dst) and not rewrite_if_exist):
        np.save(dst, records_Y)

## For ablation studies

In [72]:
destination_dir = "/projects/mcmm/ablation_final/"
rewrite_if_exist = False

In [73]:
mcir_dir = "/projects/mcmm/benchmark_2023-05-13/Ackley_May12/"

In [74]:
for filename in os.listdir(mcir_dir):
    if not filename.endswith('.npy'):
        continue
    
    filename_noext = filename.strip('.npy')
    splitted = filename_noext.split("_")
    function = splitted[0]
    dimension = int(splitted[1][:-1])
    if dimension != 50:
        continue
    identifier = splitted[2]
    study_type = "original"
    
    target_filename = "_".join([
        function,
        str(dimension),
        study_type,
        identifier,
        "Y"
    ]) + ".npy"
    
    records_Y = np.load(os.path.join(mcir_dir, filename))[:,0]
    
    dst = os.path.join(destination_dir, target_filename)
    if not (os.path.exists(dst) and not rewrite_if_exist):
        np.save(dst, records_Y)
    print(dst)

/projects/mcmm/ablation_final/Ackley_50_original_qv119de9o2lzsbqhts5f4v8u65iidxs9y8izjj4q_Y.npy
/projects/mcmm/ablation_final/Ackley_50_original_t83js8othg4vmkuxjyw68gnsq3diqznj8uzoqkoq_Y.npy
/projects/mcmm/ablation_final/Ackley_50_original_sz0rgr48vkq3tpr44ir92hyrdm6spql9vla63l0r_Y.npy
/projects/mcmm/ablation_final/Ackley_50_original_vze8wf9ub9mum1ojovb5tat52tq6n9minh3r19pg_Y.npy
/projects/mcmm/ablation_final/Ackley_50_original_so70mtvu5h4sfcq7njr9d928kl1fxkpvk7t66cix_Y.npy


In [75]:
ablation_dir = "/projects/mcmm/ablation_Ackley/"

In [76]:
for filename in os.listdir(ablation_dir):
    if not filename.endswith('.npy'):
        continue
    
    filename_noext = filename.split('.')[0]
    
    splitted = filename_noext.split("_")
    function = splitted[0]
    dimension = int(splitted[1][:-1])
    assert dimension == 50
    
    identifier = splitted[-1]
    
    study_type = filename_noext.rsplit('_', 1)[0].split('_', 4)[-1]
    study_type = study_type.replace("_", "-")
    
    target_filename = "_".join([
        function,
        str(dimension),
        study_type,
        identifier,
        "Y"
    ]) + ".npy"
    
    src = os.path.join(ablation_dir, filename)
    dst = os.path.join(destination_dir, target_filename)
    if not (os.path.exists(dst) and not rewrite_if_exist):
        shutil.copy2(src, dst)
    print(dst)

/projects/mcmm/ablation_final/Ackley_50_nodeexphigh_2_Y.npy
/projects/mcmm/ablation_final/Ackley_50_no-learning_1_Y.npy
/projects/mcmm/ablation_final/Ackley_50_nodeexphigh_3_Y.npy
/projects/mcmm/ablation_final/Ackley_50_nodeexphigh_0_Y.npy
/projects/mcmm/ablation_final/Ackley_50_lblow_3_Y.npy
/projects/mcmm/ablation_final/Ackley_50_nodeexplow_0_Y.npy
/projects/mcmm/ablation_final/Ackley_50_no-local-opt_4_Y.npy
/projects/mcmm/ablation_final/Ackley_50_boxhigh_4_Y.npy
/projects/mcmm/ablation_final/Ackley_50_explow_3_Y.npy
/projects/mcmm/ablation_final/Ackley_50_lbhigh_2_Y.npy
/projects/mcmm/ablation_final/Ackley_50_local-opt_9_Y.npy
/projects/mcmm/ablation_final/Ackley_50_boxlow_3_Y.npy
/projects/mcmm/ablation_final/Ackley_50_boxhigh_0_Y.npy
/projects/mcmm/ablation_final/Ackley_50_nodeexplow_1_Y.npy
/projects/mcmm/ablation_final/Ackley_50_lbhigh_3_Y.npy
/projects/mcmm/ablation_final/Ackley_50_local-opt_6_Y.npy
/projects/mcmm/ablation_final/Ackley_50_nodeexphigh_4_Y.npy
/projects/mcmm/abla

In [77]:
ablation_dir = "/projects/mcmm/ablation_Michalewicz/"

In [78]:
for filename in os.listdir(ablation_dir):
    if not filename.endswith('.npy'):
        continue
    
    filename_noext = filename.split('.')[0]
    
    splitted = filename_noext.split("_")
    function = splitted[0]
    dimension = int(splitted[1][:-1])
    assert dimension == 50
    
    identifier = splitted[-1]
    
    study_type = filename_noext.rsplit('_', 1)[0].split('_', 4)[-1]
    study_type = study_type.replace("_", "-")
    
    if study_type == 'baseline':
        study_type = 'original'
        
    target_filename = "_".join([
        function,
        str(dimension),
        study_type,
        identifier,
        "Y"
    ]) + ".npy"
    
    src = os.path.join(ablation_dir, filename)
    dst = os.path.join(destination_dir, target_filename)
    if not (os.path.exists(dst) and not rewrite_if_exist):
        shutil.copy2(src, dst)
    print(dst)

/projects/mcmm/ablation_final/Michalewicz_50_no-learning_3_Y.npy
/projects/mcmm/ablation_final/Michalewicz_50_no-local-opt_4_Y.npy
/projects/mcmm/ablation_final/Michalewicz_50_original_2_Y.npy
/projects/mcmm/ablation_final/Michalewicz_50_original_3_Y.npy
/projects/mcmm/ablation_final/Michalewicz_50_no-learning_2_Y.npy
/projects/mcmm/ablation_final/Michalewicz_50_nodeexphigh_1_Y.npy
/projects/mcmm/ablation_final/Michalewicz_50_nodeexplow_1_Y.npy
/projects/mcmm/ablation_final/Michalewicz_50_original_1_Y.npy
/projects/mcmm/ablation_final/Michalewicz_50_nodeexphigh_2_Y.npy
/projects/mcmm/ablation_final/Michalewicz_50_nodeexplow_4_Y.npy
/projects/mcmm/ablation_final/Michalewicz_50_no-learning_0_Y.npy
/projects/mcmm/ablation_final/Michalewicz_50_no-learning_1_Y.npy
/projects/mcmm/ablation_final/Michalewicz_50_nodeexphigh_0_Y.npy
/projects/mcmm/ablation_final/Michalewicz_50_no-learning_4_Y.npy
/projects/mcmm/ablation_final/Michalewicz_50_original_4_Y.npy
/projects/mcmm/ablation_final/Michalew

In [80]:
ablation_dir = "/projects/mcmm/ablation_Watson/"

In [92]:
for filename in os.listdir(ablation_dir):
    if not filename.endswith('.npy'):
        continue
    
    filename_noext = filename.split('.')[0]
    
    splitted = filename_noext.split("_")
    function = splitted[0]
    dimension = int(splitted[1][:-1])
#     assert dimension == 50
    
    identifier = splitted[-1]
    
    study_type = filename_noext.rsplit('_', 1)[0].split('_', 4)[-1]
    study_type = study_type.replace("_", "-")
    
    if study_type == 'baseline':
        study_type = 'original'
        
    target_filename = "_".join([
        function,
        str(dimension),
        study_type,
        identifier,
        "Y"
    ]) + ".npy"
    
#     src = os.path.join(ablation_dir, filename)
#     dst = os.path.join(destination_dir, target_filename)
#     if not (os.path.exists(dst) and not rewrite_if_exist):
#         shutil.copy2(src, dst)
#     print(dst)
    
    
    records_Y = np.load(src)[:,0]
    
    dst = os.path.join(destination_dir, target_filename)
    print(dst)
    if not (os.path.exists(dst) and not rewrite_if_exist):
        np.save(dst, records_Y)

/projects/mcmm/ablation_final/Watson_32_local-opt_0_Y.npy


## Runtime Stats

In [14]:
stats_destination_dir = "/projects/mcmm/results_final/stats"

In [1]:
# baselines

In [2]:
baseline_dir_1 = "/projects/mcmm/running_zhizhen_exps/baseline_results_displacement/record"
baseline_dir_2 = "/projects/mcmm/running_zhizhen_exps/baseline_results_displacement_more_iterations/record"

In [7]:
baseline_stats_dir_1 = "/projects/mcmm/running_zhizhen_exps/baseline_results_displacement/results/"
baseline_stats_dir_2 = "/projects/mcmm/running_zhizhen_exps/baseline_results_displacement_more_iterations/results/"

In [6]:
result_files_1 = [fn for fn in os.listdir(baseline_dir_1) if fn.endswith("Y.npy")]
result_files_2 = [fn for fn in os.listdir(baseline_dir_2) if fn.endswith("Y.npy")]

In [20]:
stats_file_1 = [fn for fn in os.listdir(baseline_stats_dir_1) if fn.endswith(".json")]
stats_file_2 = [fn for fn in os.listdir(baseline_stats_dir_2) if fn.endswith(".json")]

In [21]:
files_in_both = []

for file_1 in stats_file_1:
    if file_1 not in stats_file_2:
        dst = os.path.join(stats_destination_dir, file_1)
        if not (os.path.exists(dst) and not rewrite_if_exist):
            print("Writing to {}".format(dst))
            shutil.copy2(
                os.path.join(baseline_stats_dir_1, file_1),
                dst
            )
    else:
        files_in_both.append(file_1)
    
for file_2 in stats_file_2:
    if file_2 not in stats_file_1:
        dst = os.path.join(stats_destination_dir, file_2)
        if not (os.path.exists(dst) and not rewrite_if_exist):
            print("Writing to {}".format(dst))
            shutil.copy2(
                os.path.join(baseline_stats_dir_2, file_2),
                dst,
            )

Writing to /projects/mcmm/results_final/stats/Ackley_10_scipy_dual_annealing_0.json
Writing to /projects/mcmm/results_final/stats/Ackley_100_scipy_dual_annealing_1.json
Writing to /projects/mcmm/results_final/stats/Ackley_200_scipy_dual_annealing_4.json
Writing to /projects/mcmm/results_final/stats/Levy_200_gurobi_4.json
Writing to /projects/mcmm/results_final/stats/Michalewicz_100_scipy_dual_annealing_4.json
Writing to /projects/mcmm/results_final/stats/Levy_200_scipy_dual_annealing_4.json
Writing to /projects/mcmm/results_final/stats/Michalewicz_50_scipy_shgo_4.json
Writing to /projects/mcmm/results_final/stats/Michalewicz_100_scipy_dual_annealing_1.json
Writing to /projects/mcmm/results_final/stats/Michalewicz_200_gurobi_4.json
Writing to /projects/mcmm/results_final/stats/Ackley_50_scipy_dual_annealing_3.json
Writing to /projects/mcmm/results_final/stats/Ackley_50_gurobi_1.json
Writing to /projects/mcmm/results_final/stats/Levy_100_gurobi_0.json
Writing to /projects/mcmm/results_fi

Writing to /projects/mcmm/results_final/stats/Levy_200_scipy_simulated_annealing_3.json
Writing to /projects/mcmm/results_final/stats/Michalewicz_200_scipy_basinhopping_4.json
Writing to /projects/mcmm/results_final/stats/Ackley_100_scipy_simulated_annealing_3.json
Writing to /projects/mcmm/results_final/stats/Levy_100_scipy_simulated_annealing_2.json
Writing to /projects/mcmm/results_final/stats/Michalewicz_10_scipy_simulated_annealing_2.json
Writing to /projects/mcmm/results_final/stats/Ackley_10_scipy_simulated_annealing_2.json
Writing to /projects/mcmm/results_final/stats/Michalewicz_10_scipy_simulated_annealing_4.json
Writing to /projects/mcmm/results_final/stats/Ackley_10_scipy_simulated_annealing_3.json
Writing to /projects/mcmm/results_final/stats/Levy_200_scipy_simulated_annealing_0.json
Writing to /projects/mcmm/results_final/stats/Michalewicz_50_scipy_simulated_annealing_4.json
Writing to /projects/mcmm/results_final/stats/Michalewicz_50_scipy_simulated_annealing_2.json
Writ

In [22]:
for file in files_in_both:
    npy_file = file.strip(".json") + "_Y.npy"
    path_1 = os.path.join(baseline_dir_1, npy_file)
    path_2 = os.path.join(baseline_dir_2, npy_file)
    
    array_1 = np.load(path_1)
    array_2 = np.load(path_2)
    
    assert len(array_1.shape) == 1 and len(array_2.shape) == 1
    
    if len(array_1) > len(array_2):
        src = os.path.join(baseline_stats_dir_1, file)
    else:
        src = os.path.join(baseline_stats_dir_2, file)
    
    dst = os.path.join(stats_destination_dir, file)
    if not (os.path.exists(dst) and not rewrite_if_exist):
        print("Writing to {}".format(dst))
        shutil.copy2(src, dst)

Writing to /projects/mcmm/results_final/stats/Michalewicz_100_scipy_differential_evolution_0.json
Writing to /projects/mcmm/results_final/stats/Ackley_50_scipy_differential_evolution_3.json
Writing to /projects/mcmm/results_final/stats/Michalewicz_10_scipy_basinhopping_4.json
Writing to /projects/mcmm/results_final/stats/Michalewicz_10_scipy_differential_evolution_3.json
Writing to /projects/mcmm/results_final/stats/Ackley_200_scipy_basinhopping_4.json
Writing to /projects/mcmm/results_final/stats/Ackley_100_scipy_basinhopping_0.json
Writing to /projects/mcmm/results_final/stats/Ackley_100_scipy_differential_evolution_2.json
Writing to /projects/mcmm/results_final/stats/Ackley_10_scipy_direct_2.json
Writing to /projects/mcmm/results_final/stats/Ackley_10_scipy_direct_0.json
Writing to /projects/mcmm/results_final/stats/Michalewicz_100_scipy_differential_evolution_1.json
Writing to /projects/mcmm/results_final/stats/Ackley_200_scipy_direct_1.json
Writing to /projects/mcmm/results_final/

Writing to /projects/mcmm/results_final/stats/Michalewicz_100_scipy_basinhopping_2.json
Writing to /projects/mcmm/results_final/stats/Michalewicz_100_scipy_differential_evolution_4.json
Writing to /projects/mcmm/results_final/stats/Ackley_200_scipy_differential_evolution_3.json
Writing to /projects/mcmm/results_final/stats/Ackley_50_scipy_basinhopping_1.json
Writing to /projects/mcmm/results_final/stats/Ackley_200_scipy_differential_evolution_4.json
Writing to /projects/mcmm/results_final/stats/Michalewicz_100_scipy_direct_4.json
Writing to /projects/mcmm/results_final/stats/Ackley_10_scipy_direct_3.json
Writing to /projects/mcmm/results_final/stats/Levy_10_scipy_basinhopping_2.json
Writing to /projects/mcmm/results_final/stats/Levy_200_scipy_basinhopping_0.json
Writing to /projects/mcmm/results_final/stats/Levy_50_scipy_direct_0.json
Writing to /projects/mcmm/results_final/stats/Levy_10_scipy_differential_evolution_1.json
Writing to /projects/mcmm/results_final/stats/Ackley_50_scipy_d

In [14]:
# nlp baselines

In [23]:
baseline_stats_dir_nlp = "/projects/mcmm/baselines_nlp/results/"

In [24]:
stats_files = [fn for fn in os.listdir(baseline_stats_dir_nlp) if fn.endswith(".json")]

In [26]:
for file in stats_files:
    dst = os.path.join(stats_destination_dir, file)
    if not (os.path.exists(dst) and not rewrite_if_exist):
        print("Writing to {}".format(dst))
        shutil.copy2(
            os.path.join(baseline_stats_dir_nlp, file),
            dst
        )

Writing to /projects/mcmm/results_final/stats/Watson_32_scipy_dual_annealing_2.json
Writing to /projects/mcmm/results_final/stats/Watson_32_scipy_direct_4.json
Writing to /projects/mcmm/results_final/stats/Biggsbi1_1001_scipy_direct_2.json
Writing to /projects/mcmm/results_final/stats/Watson_32_scipy_dual_annealing_4.json
Writing to /projects/mcmm/results_final/stats/Watson_32_scipy_basinhopping_0.json
Writing to /projects/mcmm/results_final/stats/Watson_32_scipy_differential_evolution_2.json
Writing to /projects/mcmm/results_final/stats/Watson_32_scipy_basinhopping_1.json
Writing to /projects/mcmm/results_final/stats/Watson_32_scipy_shgo_0.json
Writing to /projects/mcmm/results_final/stats/Watson_32_scipy_differential_evolution_0.json
Writing to /projects/mcmm/results_final/stats/Watson_32_scipy_differential_evolution_3.json
Writing to /projects/mcmm/results_final/stats/Harkerp_101_scipy_dual_annealing_1.json
Writing to /projects/mcmm/results_final/stats/Biggsbi1_1001_scipy_differenti

In [23]:
# baseline Neural Network

In [24]:
baseline_dir_nn = "/projects/mcmm/running_zhizhen_exps/baseline_results_nn_2/record"

In [27]:
baseline_stats_dir_nn = "/projects/mcmm/running_zhizhen_exps/baseline_results_nn_2/results/"

In [35]:
def process_baseline_stats_nn_filename(filename):
    # Example: NeuralNetworkOneLayer_-1_scipy_dual_annealing_4-nn_one_layer_michalewicz_50_16.json
    filename_no_json = filename.strip('.json')
    
    nn_config = filename_no_json.rsplit('-', 1)[1]
    splitted_nn_config = nn_config.split('_')
    
    simulated_func = splitted_nn_config[3]
    input_dimension = int(splitted_nn_config[4])
    hidden_dimension = int(splitted_nn_config[5])
    
    run_info = filename_no_json.rsplit('-', 1)[0]
    splitted_run_info = run_info.split('_')
    
    seed = int(run_info.rsplit('_', 1)[1])
    identifier = str(seed)
    
    algorithm = run_info.rsplit('_', 1)[0].split('_', 3)[-1]
    
    new_file_name = '_'.join([
        f"nn-{simulated_func}",
        str(input_dimension),
        algorithm,
        identifier,
    ]) + ".json"
    
    return new_file_name

In [36]:
process_baseline_stats_nn_filename("NeuralNetworkOneLayer_-1_scipy_dual_annealing_4-nn_one_layer_michalewicz_50_16.json")

'nn-michalewicz_50_dual_annealing_4.json'

In [37]:
for file in os.listdir(baseline_stats_dir_nn):
    if not file.endswith(".json"):
        continue
    
    src = os.path.join(baseline_stats_dir_nn, file)
    new_filename = process_baseline_stats_nn_filename(file)
    dst = os.path.join(stats_destination_dir, new_filename)

    if not (os.path.exists(dst) and not rewrite_if_exist):
        print("Writing to {}".format(dst))
        shutil.copy2(src, dst)

Writing to /projects/mcmm/results_final/stats/nn-michalewicz_100_basinhopping_2.json
Writing to /projects/mcmm/results_final/stats/nn-michalewicz_50_basinhopping_0.json
Writing to /projects/mcmm/results_final/stats/nn-ackley_50_dual_annealing_3.json
Writing to /projects/mcmm/results_final/stats/nn-michalewicz_100_differential_evolution_4.json
Writing to /projects/mcmm/results_final/stats/nn-michalewicz_100_direct_3.json
Writing to /projects/mcmm/results_final/stats/nn-ackley_50_differential_evolution_1.json
Writing to /projects/mcmm/results_final/stats/nn-michalewicz_100_dual_annealing_3.json
Writing to /projects/mcmm/results_final/stats/nn-ackley_100_dual_annealing_4.json
Writing to /projects/mcmm/results_final/stats/nn-michalewicz_100_differential_evolution_0.json
Writing to /projects/mcmm/results_final/stats/nn-ackley_50_dual_annealing_0.json
Writing to /projects/mcmm/results_final/stats/nn-michalewicz_100_direct_0.json
Writing to /projects/mcmm/results_final/stats/nn-michalewicz_10

# For AAAI

In [4]:
destination_dir = "/home/ubuntu/s3/results_final/"

In [5]:
def process_aaai_baseline_filename(filename):
    # Example: lamcts_Michalewicz_d50_iter1.npy
#     print(filename)
    cand_funcs = set(["Ackley", "Levy", "Michalewicz"])
    for cand_func in cand_funcs:
        if cand_func in filename:
            idx_func_name_end = filename.find(cand_func) + len(cand_func)
            if filename[idx_func_name_end] != '_':
                filename = filename[:idx_func_name_end] + '_d' + filename[idx_func_name_end:]
        
    filename_no_npy = filename.strip('.npy').lower()
    
    splitted_config = filename_no_npy.split('_')
    
    algorithm = splitted_config[0]
    func_name = splitted_config[1]
    assert splitted_config[2][0] == 'd'
    dimension = int(splitted_config[2][1:])
    
    identifier = splitted_config[-1]
    
    new_file_name = '_'.join([
        func_name,
        str(dimension),
        algorithm,
        identifier,
        "Y"
    ]) + ".npy"
    
    return new_file_name

In [6]:
def process_aaai_nlp_filename(filename):
    # Example: lamcts_Harkerp_iter0.npy
    cand_funcs = set(["Harkerp", "Biggsbi1", "Watson"])
    for cand_func in cand_funcs:
        if cand_func in filename:
            idx_func_name_end = filename.find(cand_func) + len(cand_func)
            if filename[idx_func_name_end] != '_':
                filename = filename[:idx_func_name_end] + '_d' + filename[idx_func_name_end:]
                return process_aaai_baseline_filename(filename)
    
    func_to_dimension = {
        "harkerp": 101,
        "biggsbi1": 1001,
        "watson": 32,
    }
    
    filename_no_npy = filename.strip('.npy').lower()
    splitted_config = filename_no_npy.split('_')
    
    algorithm = splitted_config[0]
    func_name = splitted_config[1]
    assert func_name in func_to_dimension
    dimension = func_to_dimension[func_name]
    
    identifier = splitted_config[-1]
    
    new_file_name = '_'.join([
        func_name,
        str(dimension),
        algorithm,
        identifier,
        "Y"
    ]) + ".npy"
    
    return new_file_name

In [7]:
def process_aaai_nn_filename(filename):
    # Example: lamcts_nn_one_layer_michalewicz_50_16_iter4.npy
    filename_no_npy = filename.strip('.npy').lower()
    splitted_config = filename_no_npy.split('_')
    
    algorithm = splitted_config[0]
    simulated_func = splitted_config[4]
    input_dimension = int(splitted_config[5])
    
    identifier = splitted_config[-1]
    
    new_file_name = '_'.join([
        f"nn-{simulated_func}",
        str(input_dimension),
        algorithm,
        identifier,
        "Y"
    ]) + ".npy"
    return new_file_name

In [8]:
def process_aaai_filename(filename):
    # Example: lamcts_Michalewicz_d50_iter1.npy
    # Example: lamcts_Harkerp_iter0.npy
    # Example: lamcts_nn_one_layer_michalewicz_50_16_iter4.npy
    
    nlp_functions = ["watson", "harkerp", "biggsbi1"]
    if any([nlp_func in filename.lower() for nlp_func in nlp_functions]):
        return process_aaai_nlp_filename(filename)
    if "nn_one_layer" in filename:
        return process_aaai_nn_filename(filename)
    
    return process_aaai_baseline_filename(filename)

In [9]:
aaai_dirs = [
    "/home/ubuntu/s3/lamcts/",
    "/home/ubuntu/s3/CMA/",
    "/home/ubuntu/s3/turbo/",
]

In [18]:
for aaai_dir in aaai_dirs:
    print("Processing {}".format(aaai_dir))
    
    data_files = sorted([fn for fn in os.listdir(aaai_dir) if fn.endswith('.npy')])
    for filename in data_files:
        src = os.path.join(aaai_dir, filename)
        output_filename = process_aaai_filename(filename)
        dst = os.path.join(destination_dir, output_filename)
    
        if not (os.path.exists(dst) and not rewrite_if_exist):
            print("Writing to {}".format(dst))
            shutil.copy2(src, dst)

    print()

Processing /home/ubuntu/s3/lamcts/
Writing to /home/ubuntu/s3/results_final/ackley_100_lamcts_iter0_Y.npy
Writing to /home/ubuntu/s3/results_final/ackley_100_lamcts_iter1_Y.npy
Writing to /home/ubuntu/s3/results_final/ackley_100_lamcts_iter2_Y.npy
Writing to /home/ubuntu/s3/results_final/ackley_100_lamcts_iter3_Y.npy
Writing to /home/ubuntu/s3/results_final/ackley_100_lamcts_iter4_Y.npy
Writing to /home/ubuntu/s3/results_final/ackley_200_lamcts_iter0_Y.npy
Writing to /home/ubuntu/s3/results_final/ackley_200_lamcts_iter1_Y.npy
Writing to /home/ubuntu/s3/results_final/ackley_200_lamcts_iter2_Y.npy
Writing to /home/ubuntu/s3/results_final/ackley_200_lamcts_iter3_Y.npy
Writing to /home/ubuntu/s3/results_final/ackley_200_lamcts_iter4_Y.npy
Writing to /home/ubuntu/s3/results_final/ackley_50_lamcts_iter0_Y.npy
Writing to /home/ubuntu/s3/results_final/ackley_50_lamcts_iter1_Y.npy
Writing to /home/ubuntu/s3/results_final/ackley_50_lamcts_iter2_Y.npy
Writing to /home/ubuntu/s3/results_final/ackl

Writing to /home/ubuntu/s3/results_final/levy_50_cma_iter3_Y.npy
Writing to /home/ubuntu/s3/results_final/levy_50_cma_iter4_Y.npy
Writing to /home/ubuntu/s3/results_final/michalewicz_100_cma_iter0_Y.npy
Writing to /home/ubuntu/s3/results_final/michalewicz_100_cma_iter1_Y.npy
Writing to /home/ubuntu/s3/results_final/michalewicz_100_cma_iter2_Y.npy
Writing to /home/ubuntu/s3/results_final/michalewicz_100_cma_iter3_Y.npy
Writing to /home/ubuntu/s3/results_final/michalewicz_100_cma_iter4_Y.npy
Writing to /home/ubuntu/s3/results_final/michalewicz_200_cma_iter0_Y.npy
Writing to /home/ubuntu/s3/results_final/michalewicz_200_cma_iter1_Y.npy
Writing to /home/ubuntu/s3/results_final/michalewicz_200_cma_iter2_Y.npy
Writing to /home/ubuntu/s3/results_final/michalewicz_200_cma_iter3_Y.npy
Writing to /home/ubuntu/s3/results_final/michalewicz_200_cma_iter4_Y.npy
Writing to /home/ubuntu/s3/results_final/michalewicz_50_cma_iter0_Y.npy
Writing to /home/ubuntu/s3/results_final/michalewicz_50_cma_iter1_Y.